In [233]:
import numpy as np
%matplotlib qt
import matplotlib.pyplot as pl

In [234]:
class State:
    def __init__(self,L:int,p:float,q:int,all_up=False) -> None:
        assert L%2 == 0
        self.p = p
        self.q = q
        self.L = L
        # index = np.arange(0,2**L,1)
        self.data = np.zeros((2,)*L)
        if not all_up:
            self.data[(0,)*L] = 1
        if all_up:
            self.data[(1,)*L] = 1
        self.all_up = all_up
        ##
        # no_of_ones = np.zeros(2**L,dtype=int)
        # for i in range(L):
        #     no_of_ones += temp%2
        #     temp = (temp/2).astype(int)
        # self.data = (p**2)**(L-no_of_ones)*((1 - p**2)/(q**2))**no_of_ones
        # self.data = self.data.reshape((2,)*L)

In [235]:
def eventransfer(State,U_t,U_k):
    L = State.L
    p = State.p
    q = State.q
    os = State.data.reshape((4,)*(L//2))
    for x in range(L//2):
        if x%2 == 0:
            U = U_t
        else:
            U = U_k
        os = np.tensordot(U,os,axes=(-1,x))
    os = np.reshape(os,(2,)*L)
    State.data = os


def oddtransfer(State,U_t,U_k):
    L = State.L
    p = State.p
    q = State.q
    os = State.data
    os = np.moveaxis(os,0,-1) #bringing 1st spin to the last position
    os = np.reshape(os,(4,)*(L//2))

    for x in range(L//2):
        if x%2 == 0:
            U = U_t
        else:
            U = U_k
        os = np.tensordot(U,os,axes=(-1,x))
    os = np.reshape(os,(2,)*L)
    os = np.moveaxis(os,-1,0)
    State.data = os

In [246]:
def no_of_down_spins(N):
    temp = np.arange(0,2**N,1,dtype=int)
    no_of_ones = np.zeros(2**N,dtype=float)
    for i in range(N):
        no_of_ones += temp%2
        temp = (temp/2).astype(int)
    return N - no_of_ones

def TopLayer(state,no_of_downs):
    """
    This function contracts the evolved state with the top layer.
    """

    ## Caluclate no. of down spins in each configuration
    # no_of_downs = no_of_down_spins(state.L)
    fac = np.sum(state.data.reshape(2**state.L)*((state.q)**(no_of_downs-(state.L)//2)))
    
    return np.log(fac)

In [247]:
def get_U_t(p,q):
    U = np.zeros((4,4))
    U[0,0] = p**2
    U[3,0] = (1-p**2)/q**2
    U[3,3] = 1
    U[0,3] = 0
    U[3,1] = q*p**2/(q**2+1) + (1-p**2)/q
    U[3,2] = q*p**2/(q**2+1) + (1-p**2)/q
    U[0,1] = p**2*q/(q**2+1)
    U[0,2] = p**2*q/(q**2+1)
    return U

def get_U_k(p,q):
    U = np.zeros((4,4))
    U[0,0] = 1
    U[3,0] = 0
    U[3,3] = p**2
    U[0,3] = (1-p**2)/q**2
    U[0,2] = q*p**2/(q**2+1) + (1-p**2)/q
    U[0,1] = q*p**2/(q**2+1) + (1-p**2)/q
    U[3,2] = p**2*q/(q**2+1)
    U[3,1] = p**2*q/(q**2+1)
    return U

In [280]:
def free_energy(L,p,q,T,all_up=False):

    state = State(L,p,q,all_up)
    log_Z = []
    top_layer_factor = []
    ##
    U_t = get_U_t(p,q)
    U_k = get_U_k(p,q)
    seed=1
    rng = np.random.default_rng(seed=seed)
    no_of_downs = no_of_down_spins(state.L)
    ##

    for t in range(T):
        
        if t%2 == 0:
            eventransfer(state,U_t,U_t)
        else:
            oddtransfer(state,U_k,U_k)
        sd = np.sum(state.data)
        log_Z.append(np.log(sd))
        state.data = state.data/sd

    top_layer_factor.append(TopLayer(state,no_of_downs))
    
FileNotFoundError
    return log_Z,top_layer_factor,state


In [286]:
F = {}
F_t = {}
for L in [8,10,12,14,16,18][:]:
    p_list = np.round(np.linspace(0.5,1,10),2)
    T = 10*L
    q = 2
    F[L] = []
    F_t[L] = {}
    for p in p_list:
        F_t[L][p] = []
        log_Z_up,top_layer_up,_ = free_energy(L,p,q,T,all_up=True)
        log_Z_down, top_layer_down,_ = free_energy(L,p,q,T,all_up=False)
        for t in [T-1]:
            temp_up = np.sum(log_Z_up[:t+1]) + top_layer_up[-1]
            temp_down = np.sum(log_Z_down[:t+1]) + top_layer_down[-1]
            F_t[L][p].append(temp_down-temp_up)
        F[L].append(F_t[L][p][-1])
            

In [283]:
for L in [14]:
    for p in F_t[L]:
        T_data = np.arange(1,len(F_t[L][p])+1,1)[::2]/L
        pl.plot(T_data,1+np.array(F_t[L][p])[::2]/(L*np.log(q)),'-^',label = r'$p=$'+str(p))
pl.legend()

In [293]:
for L in F:
    pl.plot(p_list,[1+np.array(F_t[L][p][-1])/(L*np.log(q)) for p in p_list],'-o',label = r'$L=$'+str(L))
pl.ylabel(r'$\Delta F$',fontsize=16)
pl.xlabel(r'$p$',fontsize=16)
pl.title(r'Haar random dynamics, transfer_matrix. $q=2,T=10L$,' +'\n'+ '$p_O=p_E=0.5$',fontsize=16)
pl.legend(fontsize=16)
pl.tight_layout()

In [242]:
p_list

array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])